In [1]:
import csv

In [2]:
with open('./data/converted_crime_data/Crime_pass5_no_tracts.csv') as f:
    data = [r for r in csv.reader(f)]
with open('./data/converted_crime_data/Crime_pass6_locations.csv') as f:
    locations = [r for r in csv.reader(f)]

In [3]:
list(enumerate(data[0])), list(enumerate(locations[0]))

([(0, 'address'),
  (1, 'apt'),
  (2, 'case'),
  (3, 'primary_offense'),
  (4, 'offense_code'),
  (5, 'offense'),
  (6, 'date_rept'),
  (7, 'date_occu'),
  (8, 'hour_occu'),
  (9, 'date_fnd'),
  (10, 'hour_fnd'),
  (11, 'fid'),
  (12, 'x'),
  (13, 'y')],
 [(0, 'address'), (1, 'fid'), (2, 'building_name'), (3, 'x'), (4, 'y')])

In [4]:
len(data), len(set((r[12], r[13]) for r in data[1:])), len(set(r[0] for r in data[1:]))

(8138, 572, 442)

number human readable locations < number locations << number crimes

In [5]:
from collections import Counter

In [6]:
sorted(Counter(r[0] for r in data[1:]).items(), key=lambda tup: tup[1])[-10:]

[('1241 THEO DICKINSON DR', 225),
 ('1239 DICKINSON DR', 291),
 ('5185 PONCE DE LEON BLVD', 323),
 ('1239 THEO DICKINSON DR', 331),
 ('5665 PONCE DE LEON BLVD', 342),
 ('1101 STANFORD DR', 343),
 ('1306 STANFORD DR', 354),
 ('1300 MEMORIAL DR', 358),
 ('1231 DICKINSON DR', 390),
 ('1231 THEO DICKINSON DR', 451)]

only use the human readable location (`r[0]`) as a last resort...many crimes are reported at 5665 Ponce de Leon Blvd, the police station but with a far more usable lat, lon value

## Other frequent addresses 

http://www.miami.edu/sa/index.php/residential_life/services/mail_packages/

* Eaton Residential College	1211 Dickinson Drive
* Mahoney Residential College	1101 Stanford Drive
* Pearson Residential College	5185 Ponce de Leon Blvd.
* Hecht Residential College 	1231 Dickinson Drive
* Stanford Residential College	1239 Dickinson Drive
* University Village	1527 Albenga Avenue

# Use `locations.csv`

Now, we'll join the data in `locations.csv` onto `Crime_pass5*.csv` using the address column.

In [7]:
# I forgot to check for x, y == 0, 0...null empty string while cleaning up anyway
for row in data[1:]:
    if row[12] == '' or row[13] == '' or abs(float(row[12])) < 1 or abs(float(row[13])) < 1:
        row[12], row[13] = None, None

First, check if an address uniquely identifies a lat, lon pair

In [8]:
# [(0, 'address'), (1, 'fid'), (2, 'building_name'), (3, 'x'), (4, 'y')]]
res = {k: v for k, v in Counter(r[0].strip() for r in locations[1:]).items() if v > 1}
res

{'SS7': 2, 'University Center': 26}

So no...makes sense for the UC since there are many subdivions

Is this even a problem in `data`?

In [9]:
data_latlon_missing = [data[0]] + [r for r in data[1:] if r[12] is None]
data_latlon_not_missing = [data[0]] + [r for r in data[1:] if r[12] is not None]

In [10]:
len(data_latlon_missing), len(data_latlon_not_missing), len(data_latlon_missing) + len(data_latlon_not_missing) - 1, len(data)

(3936, 4203, 8138, 8138)

In [11]:
sum(r for r in data_latlon_missing[1:] if r[0] in ('SS7', 'UNIVERSITY CENTER'))

0

So it's not even a problem! Just exclude those data.

## Sanitize before join

In [12]:
import re
rex1 = re.compile(r'\W+')
rex2 = re.compile(r'\.')
clean_addr = lambda s: rex1.sub(' ', rex2.sub(' ', s)).upper().strip()

In [13]:
locations_d = {clean_addr(r[0]): r[1:] for r in locations[1:] if r[0] not in set(['SS7', 'University Center'])}

In [14]:
matches = [r for r in data_latlon_missing[1:] if r[0] in locations_d]
len(matches), len(data_latlon_missing[1:])

(447, 3935)

## Join

In [15]:
data_joined_with_location = [data_latlon_missing[0]]
for r in data_latlon_missing[1:]:
    if r[12] is None and r[0] in locations_d:
        info = locations_d[r[0]]
        r[11], r[12], r[13] = info[0], info[2], info[3]
    data_joined_with_location.append(r)

In [16]:
len(data_joined_with_location)

3936

# Geolocate the rest

In [18]:
need_to_geocode = [data_joined_with_location[0]] + [r for r in data_joined_with_location[1:] if r[12] is None and r[0] != '']

In [19]:
unique_missing_addresses = set(r[0] for r in need_to_geocode[1:])
len(need_to_geocode), len(unique_missing_addresses)

(3482, 310)

So there are only 310 locations without latlon left! Let's see if we have previously cached results, then if necessary call out to the Google geocoder (this requires an API key not included). Since this is my second time around, I'll cheat a little

In [20]:
from os.path import exists as file_exists
geocode_cache_fn = './data/cached_geocode_data.csv'

In [29]:
if file_exists(geocode_cache_fn):
    with open(geocode_cache_fn) as f:
        previously_geocoded_data = [r for r in csv.reader(f)]
else:
    previously_geocoded_data = []

In [30]:
addresses_to_fetch = unique_missing_addresses - set(x[0] for x in previously_geocoded_data[1:])
if len(addresses_to_fetch) > 0:
    from geopy.geocoders import GoogleV3
    from time import sleep
    geolocator = GoogleV3(api_key='')
    geolocated = {}
    failed = []
    for loc in addresses_to_fetch:
        res = geolocator.geocode(loc + ' CORAL GABLES FL')
        if res is None:
            print('failed to geolocate {}'.format(loc))
            failed.append(loc)
        else:
            print('got {}'.format(loc))
            geolocated[loc] = res
        sleep(1 / 10) # API throttling
    with open(geocode_cache_fn, 'a') as f:
        writer = csv.writer(f)
        if not file_exists(geocode_cache_fn)
            writer.writerow(['address', 'geolocated_address', 'lat', 'lon', 'altitude'])
        for key, value in geolocated.items():
            new_row = [key, value.address, value.latitude, value.longitude, value.altitude]
            writer.writerow(new_row)
            previously_geocoded_data.append(new_row)

# Convert and replace

In [38]:
# ['address', 'geolocated_address', 'lat', 'lon', 'altitude']
previously_geocoded_data_d = {r[0]: r[1:] for r in previously_geocoded_data}

In [39]:
mapping_gcs = ['address', 'apt',
          'case', 'primary_offense',
          'offense_code', 'offense',
          'date_rept',
          'date_occu', 'hour_occu',
          'date_fnd', 'hour_fnd',
          'fid', 'lat', 'lon']
data_gcs = [mapping_gcs]

data_spfl = [list(data[0])]
for row in data_joined_with_location[1:]:
    row = list(row) # make sure we're dealing with a copy and not a pointer
    if row[12] == None:
        if row[0] != '':
            row[1], row[5] = clean_addr(row[1]), clean_addr(row[5]) 
            info = previously_geocoded_data_d[row[0]]
            row[12], row[13] = info[1], info[2]
            data_gcs.append(row)
    else:
        row[1], row[5] = clean_addr(row[1]), clean_addr(row[5])
        data_spfl.append(row)
for row in data_latlon_not_missing[1:]:
    row = list(row) # make sure we're dealing with a copy and not a pointer
    row[1], row[5] = clean_addr(row[1]), clean_addr(row[5]) 
    data_spfl.append(row)

## Remove manually seen outliers

In [42]:
data_spfl_clean = [data_spfl[0]]
for row in data_spfl[1:]:
    if float(row[12]) < 89380:
        continue
    data_spfl_clean.append(row)

In [43]:
with open('./data/converted_crime_data/Crime_pass7_latlon.csv', 'w') as f:
    writer = csv.writer(f)
    for row in data_gcs:
        writer.writerow(row)
with open('./data/converted_crime_data/Crime_pass7_spfl.csv', 'w') as f:
    writer = csv.writer(f)
    for row in data_spfl_clean:
        writer.writerow(row)